In [1]:
import numpy as np
import math
from tqdm.notebook import tnrange
# Define the parameters of the problem
S0 = 100   # initial stock price
K = 105    # strike price
r = 0.005   # risk-free interest rate
T = 1      # time to expiration
sigma = 0.3   # volatility
N = 10    # number of time steps

# Calculation of parameters
dt = 0.003
u = np.exp(sigma * np.sqrt(dt))
d = 1 / u
p = (np.exp(r * dt) - d) / (u - d)
q = 1-p
# Define the state and action spaces
stock_price = np.zeros((N + 1, N + 1))
stock_price[0, 0] = S0

for i in range(1, N + 1):
    stock_price[i, 0] = stock_price[i-1, 0] * u
    for j in range(1, i + 1):
        stock_price[i, j] = stock_price[i-1, j-1] * d

actions = np.linspace(-1,0,1001)  # hold or buy stock
actions_1 = np.linspace(0,K,1001)


In [2]:
states = []
for i in range(len(stock_price)):
    for j in range(len(stock_price[0])):
        if stock_price[i][j] !=0:
            states.append([stock_price[i][j],i])
states = np.array(states)

In [3]:
states

array([[100.        ,   0.        ],
       [101.65674192,   1.        ],
       [ 98.37025869,   1.        ],
       [103.34093178,   2.        ],
       [100.        ,   2.        ],
       [ 96.76707794,   2.        ],
       [105.05302431,   3.        ],
       [101.65674192,   3.        ],
       [ 98.37025869,   3.        ],
       [ 95.1900249 ,   3.        ],
       [106.79348181,   4.        ],
       [103.34093178,   4.        ],
       [100.        ,   4.        ],
       [ 96.76707794,   4.        ],
       [ 93.63867374,   4.        ],
       [108.56277419,   5.        ],
       [105.05302431,   5.        ],
       [101.65674192,   5.        ],
       [ 98.37025869,   5.        ],
       [ 95.1900249 ,   5.        ],
       [ 92.11260559,   5.        ],
       [110.36137918,   6.        ],
       [106.79348181,   6.        ],
       [103.34093178,   6.        ],
       [100.        ,   6.        ],
       [ 96.76707794,   6.        ],
       [ 93.63867374,   6.        ],
 

In [4]:
def Payoff(s):
    """Compute the payoff for being in state s at the terminal time"""
    return max(K-s, 0) 
def transition(s,t, a):
    """Compute the next state and reward for taking action a in state s"""
    s_u = s * u
    s_d = s * d
    Payoff_u = Payoff(s_u)
    r_u = (Payoff_u-a[0]*s_u-a[1])**2
    Payoff_d = Payoff(s_d)
    r_d = (Payoff_d-a[0]*s_d-a[1])**2
    return [(s_u,t+1,r_u, p), (s_d,t+1,r_d, q)]

In [5]:
V_new = np.zeros(len(states))
pi_new = []
for i, s in enumerate(states):
    Q = np.zeros((len(actions), len(actions_1)))
    for j, a in enumerate(actions):
            for k,b in enumerate(actions_1):
                next_states = transition(s[0],s[1],[a,b])
                if s[1]<N:
                    Q[j][k] = sum([p_next * (reward_next) for s_next,t_next,reward_next, p_next in next_states])
                else :
                    pass 
    V_new[i] = np.min(Q)
    optimal_action = np.unravel_index(Q.argmin(), Q.shape)
    pi_new.append(optimal_action)
V = V_new

In [6]:
pi_new

[(0, 1000),
 (0, 1000),
 (0, 1000),
 (0, 1000),
 (0, 1000),
 (0, 1000),
 (520, 488),
 (0, 1000),
 (0, 1000),
 (0, 1000),
 (1000, 0),
 (0, 1000),
 (0, 1000),
 (0, 1000),
 (0, 1000),
 (1000, 0),
 (520, 488),
 (0, 1000),
 (0, 1000),
 (0, 1000),
 (0, 1000),
 (1000, 0),
 (1000, 0),
 (0, 1000),
 (0, 1000),
 (0, 1000),
 (0, 1000),
 (0, 1000),
 (1000, 0),
 (1000, 0),
 (520, 488),
 (0, 1000),
 (0, 1000),
 (0, 1000),
 (0, 1000),
 (0, 1000),
 (1000, 0),
 (1000, 0),
 (1000, 0),
 (0, 1000),
 (0, 1000),
 (0, 1000),
 (0, 1000),
 (0, 1000),
 (0, 1000),
 (1000, 0),
 (1000, 0),
 (1000, 0),
 (520, 488),
 (0, 1000),
 (0, 1000),
 (0, 1000),
 (0, 1000),
 (0, 1000),
 (0, 1000),
 (0, 0),
 (0, 0),
 (0, 0),
 (0, 0),
 (0, 0),
 (0, 0),
 (0, 0),
 (0, 0),
 (0, 0),
 (0, 0),
 (0, 0)]

In [7]:
%%time
V = np.zeros(len(states))
pi = np.zeros(len(states), dtype=int) 
for t in range(T - 1, -1, -1):  # iterate backwards in time\
    V_new = np.zeros(len(states))
    pi_new = []
    for i, s in enumerate(states):
        Q = np.zeros((len(actions), len(actions_1)))
        for j, a in enumerate(actions):
                for k,b in enumerate(actions_1):
                    next_states = transition(s[0],s[1],[a,b])
                    if s[1]<N:
                        Q[j][k] = sum([p_next * (reward_next) for s_next,t_next,reward_next, p_next in next_states])
                    else :
                        pass 
        V_new[i] = np.min(Q)
        optimal_action = np.unravel_index(Q.argmin(), Q.shape)
        pi_new.append(optimal_action)
    V = V_new
    pi = np.array(pi_new)

CPU times: total: 41.2 s
Wall time: 4min 11s


In [8]:
pi

array([[   0, 1000],
       [   0, 1000],
       [   0, 1000],
       [   0, 1000],
       [   0, 1000],
       [   0, 1000],
       [ 520,  488],
       [   0, 1000],
       [   0, 1000],
       [   0, 1000],
       [1000,    0],
       [   0, 1000],
       [   0, 1000],
       [   0, 1000],
       [   0, 1000],
       [1000,    0],
       [ 520,  488],
       [   0, 1000],
       [   0, 1000],
       [   0, 1000],
       [   0, 1000],
       [1000,    0],
       [1000,    0],
       [   0, 1000],
       [   0, 1000],
       [   0, 1000],
       [   0, 1000],
       [   0, 1000],
       [1000,    0],
       [1000,    0],
       [ 520,  488],
       [   0, 1000],
       [   0, 1000],
       [   0, 1000],
       [   0, 1000],
       [   0, 1000],
       [1000,    0],
       [1000,    0],
       [1000,    0],
       [   0, 1000],
       [   0, 1000],
       [   0, 1000],
       [   0, 1000],
       [   0, 1000],
       [   0, 1000],
       [1000,    0],
       [1000,    0],
       [1000,

In [9]:
V

array([0.        , 0.        , 0.        , 0.00139552, 0.        ,
       0.        , 0.00047609, 0.        , 0.        , 0.        ,
       0.        , 0.00139552, 0.        , 0.        , 0.        ,
       0.        , 0.00047609, 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.00139552, 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.00047609, 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.00139552,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.00047609, 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        ])

In [10]:
pay_off = np.array([])
delta = np.array([]) 
cash = np.array([])
for i,s in enumerate(states):
    if s[1]<N:
        delta = np.append(delta,actions[pi[i][0]])
        pay_off = np.append(pay_off,max(K-s[0],0))
        cash = np.append(cash,actions_1[pi[i][1]])

In [11]:
delta

array([-1.  , -1.  , -1.  , -1.  , -1.  , -1.  , -0.48, -1.  , -1.  ,
       -1.  ,  0.  , -1.  , -1.  , -1.  , -1.  ,  0.  , -0.48, -1.  ,
       -1.  , -1.  , -1.  ,  0.  ,  0.  , -1.  , -1.  , -1.  , -1.  ,
       -1.  ,  0.  ,  0.  , -0.48, -1.  , -1.  , -1.  , -1.  , -1.  ,
        0.  ,  0.  ,  0.  , -1.  , -1.  , -1.  , -1.  , -1.  , -1.  ,
        0.  ,  0.  ,  0.  , -0.48, -1.  , -1.  , -1.  , -1.  , -1.  ,
       -1.  ])

In [12]:
X = delta 
Y = cash
# Print the results
print("Optimal hedging portfolio (buy stock, hold cash):")
for s, x, y in zip(states, X, Y):
    print("S = %.5f: (%.2f, %.2f)" % (s[0], x, y))


Optimal hedging portfolio (buy stock, hold cash):
S = 100.00000: (-1.00, 105.00)
S = 101.65674: (-1.00, 105.00)
S = 98.37026: (-1.00, 105.00)
S = 103.34093: (-1.00, 105.00)
S = 100.00000: (-1.00, 105.00)
S = 96.76708: (-1.00, 105.00)
S = 105.05302: (-0.48, 51.24)
S = 101.65674: (-1.00, 105.00)
S = 98.37026: (-1.00, 105.00)
S = 95.19002: (-1.00, 105.00)
S = 106.79348: (0.00, 0.00)
S = 103.34093: (-1.00, 105.00)
S = 100.00000: (-1.00, 105.00)
S = 96.76708: (-1.00, 105.00)
S = 93.63867: (-1.00, 105.00)
S = 108.56277: (0.00, 0.00)
S = 105.05302: (-0.48, 51.24)
S = 101.65674: (-1.00, 105.00)
S = 98.37026: (-1.00, 105.00)
S = 95.19002: (-1.00, 105.00)
S = 92.11261: (-1.00, 105.00)
S = 110.36138: (0.00, 0.00)
S = 106.79348: (0.00, 0.00)
S = 103.34093: (-1.00, 105.00)
S = 100.00000: (-1.00, 105.00)
S = 96.76708: (-1.00, 105.00)
S = 93.63867: (-1.00, 105.00)
S = 90.61141: (-1.00, 105.00)
S = 112.18978: (0.00, 0.00)
S = 108.56277: (0.00, 0.00)
S = 105.05302: (-0.48, 51.24)
S = 101.65674: (-1.00,